In [24]:
%matplotlib inline
from segmentation.model.frame import Framework
from segmentation.data.slice import get_tiff_np, read_shp, read_tiff, get_mask
import segmentation.model.functions as fn
from segmentation.model.metrics import *

import yaml
import pathlib
import rasterio
import rasterio.plot
import torch
import pdb
from addict import Dict
import numpy as np
import pandas as pd
from skimage.color import rgb2hsv
import matplotlib.pyplot as plt
from scipy.ndimage.morphology import binary_fill_holes
from PIL import Image
from tqdm import tqdm

import seaborn_image as isns

# TODO: Update code that gives FutureWarning and DeprecationWarning
import warnings
warnings.filterwarnings("ignore")

conf = Dict(yaml.safe_load(open('./conf/predict_slices.yaml')))

#% Prediction-specific config
runs_dir = pathlib.Path(conf.runs_dir)
output_dir = pathlib.Path(conf.output_dir)

#% Load checkpoint using the training config
checkpoint_path = runs_dir / conf.run_name / 'models' / 'model_best.pt'
frame: Framework = Framework.from_checkpoint(checkpoint_path, device=conf.gpu_rank, testing=True)

#% Load data stuff
tiff_path = pathlib.Path('/home/jperez/data/HKH_raw/Landsat7_2005/')
dem_path = pathlib.Path('/home/jperez/data/HKH_raw/DEM/')
labels_fname = pathlib.Path('/home/jperez/data/HKH_raw/labels/HKH_CIDC_5basins_all.shp')
labels = read_shp(labels_fname)

UNET dropout= 1e-08


organizePolygons() received an unexpected geometry.  Either a polygon with interior rings, or a polygon with less than 4 points, or a non-Polygon geometry.  Return arguments as a collection.


In [45]:
df = pd.read_csv('pred.csv', index_col=0)

# DCI bad predictions
# If they have high number of pixels in the label but our iou is really low, we should take a look at it
# df_sorted = df.sort_values(['dci_n'], ascending=False).sort_values('dci_iou')
df_sorted = df.sort_values(['dci_n'], ascending=False)
df_sorted = df_sorted[(df_sorted['dci_iou'] <= 1e-20)]
df_sorted.iloc[:10]

,fname,total_n,bg_n,bg_n%,bg_prec,bg_rec,bg_iou,ci_n,ci_n%,ci_prec,ci_rec,ci_iou,dci_n,dci_n%,dci_prec,dci_rec,dci_iou
47,image12,3159468,3022408,0.956619,0.990531,0.973559,0.964582,119001,0.037665,0.457492,0.690249,0.379567,18059,0.005716,0.0,0.0,0.0
113,image11,3186335,3087797,0.969075,0.999668,0.971706,0.971393,86200,0.027053,0.085812,0.895954,0.084965,12338,0.003872,0.0,0.0,0.0
66,image92,2517933,2353526,0.934706,0.991920,0.969554,0.961956,152956,0.060747,0.593190,0.823975,0.526474,11451,0.004548,0.0,0.0,0.0
167,image128,2225430,2151132,0.966614,0.999517,0.972528,0.972072,65953,0.029636,0.202007,0.949947,0.199880,8345,0.003750,0.0,0.0,0.0
125,image114,2597956,2453473,0.944386,0.991282,0.979250,0.970888,138358,0.053256,0.670399,0.811235,0.579933,6125,0.002358,0.0,0.0,0.0
197,image156,3087399,3008880,0.974568,0.997449,0.978435,0.975992,72481,0.023476,0.169410,0.612541,0.153013,6038,0.001956,0.0,0.0,0.0
103,image153,3077833,2785090,0.904887,0.998071,0.909761,0.908164,287193,0.093310,0.058365,0.748437,0.057242,5550,0.001803,0.0,0.0,0.0
112,image172,2971584,2914884,0.980919,0.991580,0.983535,0.975389,51589,0.017361,0.157882,0.248619,0.106883,5111,0.001720,0.0,0.0,0.0
126,image168,2129061,1987957,0.933725,0.982925,0.974729,0.958498,136116,0.063932,0.663045,0.725560,0.530099,4988,0.002343,0.0,0.0,0.0
100,image111,105223,87912,0.835483,0.999010,0.835434,0.834743,12503,0.118824,0.000880,0.112245,0.000874,4808,0.045693,0.0,0.0,0.0


In [55]:
# DCI good predictions
df_sorted = df.sort_values(['dci_n'], ascending=False)
df_sorted = df_sorted[df_sorted['dci_iou'] >= 0.01]
df_sorted.iloc[:]

,fname,total_n,bg_n,bg_n%,bg_prec,bg_rec,bg_iou,ci_n,ci_n%,ci_prec,ci_rec,ci_iou,dci_n,dci_n%,dci_prec,dci_rec,dci_iou
152,image177,3066315,2910180,0.949081,0.974668,0.980557,0.956188,145580,0.047477,0.668059,0.683222,0.510067,10555,0.003442,0.122785,0.041456,0.031983
39,image116,2188242,1932688,0.883215,0.951384,0.928626,0.886556,246902,0.112831,0.451151,0.617519,0.352617,8652,0.003954,0.135691,0.042221,0.033273
121,image95,1080760,1010001,0.934528,0.984809,0.958602,0.944634,63998,0.059216,0.296791,0.796762,0.275903,6761,0.006256,0.067002,0.023462,0.017684
104,image8,2395678,2196186,0.916728,0.956689,0.962397,0.922216,195098,0.081437,0.586126,0.648920,0.445010,4394,0.001834,0.109012,0.013196,0.011911
174,image22,3066528,2853063,0.930389,0.950024,0.988723,0.939840,209137,0.068200,0.855784,0.657524,0.591934,4328,0.001411,0.655961,0.053630,0.052163
16,image29,3038892,2631160,0.865829,0.943108,0.976706,0.922362,403873,0.132901,0.858490,0.765348,0.679611,3859,0.001270,0.294636,0.025145,0.023717
139,image136,3023408,2767205,0.915260,0.944048,0.976546,0.923118,253524,0.083854,0.756733,0.670889,0.551867,2679,0.000886,0.279209,0.012001,0.011640
75,image119,3037830,2893994,0.952652,0.990332,0.978612,0.969352,141479,0.046572,0.559864,0.763519,0.477128,2357,0.000776,0.036487,0.015826,0.011162


In [56]:
D_split = {}
pred_dir = pathlib.Path('pred_runs/images/multi_phys64_s1/t=0.5')
all_pred_fnames = list(pred_dir.glob('*.tif'))
for pred_fname in all_pred_fnames:
    s = pred_fname.name.split('_')
    im_fname = s[0]
    split = s[1][:-4]
    D_split[im_fname] = split

for i in range(8):
    fname = pathlib.Path(df_sorted.iloc[i]['fname'] + '.tif')
    # print(fname, df_sorted.iloc[i], split)

    tiff_fname = tiff_path / fname.name
    split = D_split[fname.name[:-4]]
    pred_fname =  pred_dir/ f'{fname.stem}_{split}.tif'
    dem_fname = dem_path / fname

    assert tiff_fname.exists()
    assert pred_fname.exists()
    assert dem_fname.exists()

    im_tiff = np.transpose(read_tiff(tiff_fname).read(), (1, 2, 0)).astype(np.uint8)
    pred_tiff = np.transpose(read_tiff(pred_fname).read(), (1, 2, 0)).astype(np.uint8)
    mask = np.sum(pred_tiff[:, :, :3], axis=2) < 0.01

    label_mask = get_mask(tiff_fname, labels)
    y_true = frame.get_y_true(label_mask, mask)

    fig, ax = plt.subplots(1, 3, figsize=(15, 15))
    ax[0].set_title(fname.name)
    ax[1].set_title('pred')
    ax[2].set_title('truth')
    isns.imgplot(im_tiff[:, :, [2, 1, 0]], ax[0]);
    isns.imgplot(pred_tiff[:, :, 0], ax[1]);
    isns.imgplot(pred_tiff[:, :, 1], ax[2]);
    plt.savefig(f'output/finder/top{i+1}_{fname.stem}_{split}.png')
    plt.close(fig)

In [ ]:
df = pd.read_csv('/home/jperez/data/HKH/processed_L07_2005/slice_meta.csv')
print(np.load('/home/jperez/data/HKH/processed_L07_2005/train/tiff_103_slice_11.npy').shape)
df

In [ ]:
df = pd.read_csv('/home/jperez/data/HKH/processed_L07_2005_phys64_s1/slice_meta.csv')
print(np.load('/home/jperez/data/HKH/processed_L07_2005_phys64_s1/train/tiff_103_slice_11.npy').shape)
df